# 🛍️ Product Recommendation  UI

Interactive interface to test our trained model with new review data.

In [5]:
# Import libraries
import pandas as pd
import joblib
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Import required libraries for preprocessing classes
import spacy
from sklearn.base import BaseEstimator, TransformerMixin
from textblob import TextBlob

In [ ]:
# Define preprocessing classes
class TextPreprocessor(BaseEstimator, TransformerMixin):
    """
    Custom text preprocessing transformer for scikit-learn pipelines using spaCy
    Performs: normalization, stopword removal, tokenization, lemmatization
    """
    
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        """Transform text data through preprocessing steps"""
        return [self._preprocess_text(text) for text in X]
    
    def _preprocess_text(self, text):
        """Apply all preprocessing steps to a single text"""
        if pd.isna(text) or text == '':
            return ''
        
        # 1. Normalize text (lowercase)
        text = text.lower()
        
        # Process with spaCy
        doc = self.nlp(text)
        
        # 2, 3, 4: Tokenization, stopword removal, lemmatization in one step
        tokens = [token.lemma_ for token in doc 
                 if not token.is_stop and not token.is_punct 
                 and not token.is_space and token.is_alpha]
        
        # Return as single string for TF-IDF compatibility
        return ' '.join(tokens)

class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_new = X.copy()
        
        # Text length features
        X_new['title_length'] = X['Title'].str.len()
        X_new['review_length'] = X['Review Text'].str.len()
        X_new['title_word_count'] = X['Title'].str.split().str.len()
        X_new['review_word_count'] = X['Review Text'].str.split().str.len()
        
        # Sentiment analysis
        X_new['review_sentiment'] = X['Review Text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
        
        # Age groups 
        X_new['age_group'] = pd.cut(X['Age'], 
                                   bins=[0, 25, 35, 45, 55, 100], 
                                   labels=['18-25', '26-35', '36-45', '46-55', '55+'])
        
        # Feedback ratio (normalized by age - older customers might get more feedback)
        X_new['feedback_per_age'] = X['Positive Feedback Count'] / (X['Age'] + 1)
        
        return X_new

print("Classes defined successfully!")

Classes defined successfully!


In [ ]:
# Load model and data
try:
    model_pipeline = joblib.load('../cache/best_model_pipeline.pkl')
    df = pd.read_csv('../dataset/reviews.csv')
    
    # Create mappings for dropdowns and auto-completion
    clothing_ids = sorted(df['Clothing ID'].unique())
    divisions = sorted(df['Division Name'].unique())
    departments = sorted(df['Department Name'].unique())
    classes = sorted(df['Class Name'].unique())
    
    id_mappings = df[['Clothing ID', 'Class Name', 'Department Name', 'Division Name']].drop_duplicates()
    id_to_class = dict(zip(id_mappings['Clothing ID'], id_mappings['Class Name']))
    id_to_dept = dict(zip(id_mappings['Clothing ID'], id_mappings['Department Name']))
    id_to_div = dict(zip(id_mappings['Clothing ID'], id_mappings['Division Name']))
    
    print("✅ Setup complete! Model and data loaded successfully.")
    print(f"Dataset shape: {df.shape}")
    print(f"Available clothing IDs: {len(clothing_ids)}")
    
except Exception as e:
    print(f"❌ Error loading files: {e}")
    print("Make sure the cache directory and dataset exist.")

✅ Setup complete! Model and data loaded successfully.
Dataset shape: (18442, 9)
Available clothing IDs: 531


In [8]:
# UI styling
display(HTML("""
<style>
.widget-label { font-weight: bold; color: #2E7D32; }
.section-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 10px;
    border-radius: 8px;
    margin: 15px 0 10px 0;
    text-align: center;
    font-weight: bold;
    font-size: 16px;
}
</style>
"""))

In [9]:
# Complete Interactive UI
display(HTML('<div class="section-header">🛍️ Product Recommendation Predictor</div>'))

# Create all widgets
age_widget = widgets.IntSlider(value=35, min=18, max=99, description='Age:', style={'description_width': '150px'})

clothing_id_widget = widgets.Dropdown(options=clothing_ids, value=clothing_ids[0], description='Clothing ID:', style={'description_width': '150px'})
division_widget = widgets.Dropdown(options=divisions, value=divisions[0], description='Division:', style={'description_width': '150px'})
department_widget = widgets.Dropdown(options=departments, value=departments[0], description='Department:', style={'description_width': '150px'})
class_widget = widgets.Dropdown(options=classes, value=classes[0], description='Class:', style={'description_width': '150px'})
feedback_widget = widgets.IntSlider(value=0, min=0, max=50, description='Positive Feedback:', style={'description_width': '150px'})

title_widget = widgets.Text(value='Great product!', description='Review Title:', style={'description_width': '150px'}, layout=widgets.Layout(width='600px'))
review_widget = widgets.Textarea(value='This product exceeded my expectations. The quality is excellent and it fits perfectly.', description='Review Text:', style={'description_width': '150px'}, layout=widgets.Layout(width='600px', height='120px'))

results_output = widgets.Output()

def load_positive_example(b):
    age_widget.value = 35
    title_widget.value = "Love this dress!"
    review_widget.value = "This dress is absolutely beautiful! Perfect fit, great quality, and I received so many compliments. Definitely worth the price."
    feedback_widget.value = 5

def load_negative_example(b):
    age_widget.value = 28
    title_widget.value = "Disappointed with quality"
    review_widget.value = "The material feels cheap and the sizing is completely off. The color faded after one wash. Would not buy again."
    feedback_widget.value = 0

def make_prediction(b):
    with results_output:
        clear_output(wait=True)
        
        input_data = pd.DataFrame({
            'Clothing ID': [clothing_id_widget.value],
            'Age': [age_widget.value],
            'Title': [title_widget.value],
            'Review Text': [review_widget.value],
            'Positive Feedback Count': [feedback_widget.value],
            'Division Name': [division_widget.value],
            'Department Name': [department_widget.value],
            'Class Name': [class_widget.value]
        })
        
        prediction = model_pipeline.predict(input_data)[0]
        probability = model_pipeline.predict_proba(input_data)[0]
        
        print("🔮 PREDICTION RESULTS")
        print("=" * 50)
        
        if prediction == 1:
            print("✅ RECOMMENDATION: YES")
            print(f"🎯 Confidence: {probability[1]:.1%}")
        else:
            print("❌ RECOMMENDATION: NO")
            print(f"🎯 Confidence: {probability[0]:.1%}")
        
        print(f"\nProbabilities: Not Recommend: {probability[0]:.1%} | Recommend: {probability[1]:.1%}")

def update_product_info(change):
    clothing_id = change['new']
    if clothing_id in id_to_class:
        class_widget.value = id_to_class[clothing_id]
        department_widget.value = id_to_dept[clothing_id]
        division_widget.value = id_to_div[clothing_id]

clothing_id_widget.observe(update_product_info, names='value')

positive_btn = widgets.Button(description='😍 Positive Example', button_style='info')
negative_btn = widgets.Button(description='😞 Negative Example', button_style='danger')
predict_button = widgets.Button(description='🚀 Predict Recommendation', button_style='success')

positive_btn.on_click(load_positive_example)
negative_btn.on_click(load_negative_example)
predict_button.on_click(make_prediction)

# Display UI
complete_ui = widgets.VBox([
    age_widget,
    clothing_id_widget,
    division_widget, 
    department_widget,
    class_widget,
    feedback_widget,
    title_widget,
    review_widget,
    widgets.HBox([positive_btn, negative_btn]),
    predict_button,
    results_output
])

display(complete_ui)